# 스타벅스 매장 정보 스크래핑(전국)

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import bs4
import time
import pandas as pd

# 크롬 브라우저 실행
driver = webdriver.Chrome("C:/Users/admin/chromedriver_win32/chromedriver.exe")

#크롬 브라우저에서 스타벅스 매장 정보 싸이트에 접속하고 코드를 작성!
driver.get('https://www.starbucks.co.kr/store/store_map.do')

#싸이트 접속 후 3초간 대기하는 코드를 작성!
time.sleep(3)

# 지역 검색 버튼을 찾는 코드를 작성
loca = driver.find_element(By.CLASS_NAME, "loca_search")

# 찾은 버튼을 클릭하는 코드를 작성
loca.click()

#3초가 대기한다.
time.sleep(3)

# 시/도 정보가 위치한 태그를 찾는 코드 작성
sido = driver.find_element(By.CLASS_NAME, 'sido_arae_box')

# 시/도 정보가 위치한 태그 안의 모든 li 태그를 찾는 코드 작성
li = sido.find_elements(By.TAG_NAME, 'li')


#전국 모든 데이터를 담을 변수 생성
totalList = []

#전국의 시/도만큼 반복
for i in range(0, len(li)):
    
    # 지역 검색 버튼을 찾는 코드를 작성
    loca = driver.find_element(By.CLASS_NAME, "loca_search")

    # 찾은 버튼을 클릭하는 코드를 작성
    loca.click()

    #3초가 대기한다.
    time.sleep(3)
    
    if i == 16 : #세종시는 구군 정보가 없음
        #모든지역을 차례대로 방문
        li[i].click()

        # 3초간 대기
        time.sleep(3)
        
        # 현재 열려 있는 페이지의 모든 정보를 BeautifulSoup 형태로 변환(BeautifulSoup으로 변환하면 데이터를 가져 오지 편하다.)
        bs = bs4.BeautifulSoup(driver.page_source, 'lxml')

        #매장 이름이 있는 태그를 선택한다.(class가 loca_step3인 div태그를 선택)
        entire=bs.find('div', class_='loca_step3')

        #찾은 태그 안에서 li태그를 모두 선택한다.
        li_list = entire.findAll('li')
        
        #모든 데이터를 totalList에 추가
        totalList.extend(li_list)
    else :
        #모든지역을 차례대로 방문
        li[i].click()

        # 3초간 대기
        time.sleep(3)

        # 구/군 정보가 위치한 태그를 찾는 코드 작성
        gugun = driver.find_element(By.CLASS_NAME,'gugun_arae_box')

        # 구/군 정보가 위치한 태그 안에서 전체 구군에 대한 
        # li 태그를 찾는 코드 작성
        total_gugun = gugun.find_element(By.TAG_NAME, "li") 

        # 찾은 태그 클릭
        total_gugun.click()

        #3초간 대기
        time.sleep(3)

        # 현재 열려 있는 페이지의 모든 정보를 BeautifulSoup 형태로 변환(BeautifulSoup으로 변환하면 데이터를 가져 오지 편하다.)
        bs = bs4.BeautifulSoup(driver.page_source, 'lxml')

        #매장 이름이 있는 태그를 선택한다.(class가 loca_step3인 div태그를 선택)
        entire=bs.find('div', class_='loca_step3')

        #찾은 태그 안에서 li태그를 모두 선택한다.
        li_list = entire.findAll('li')

        #모든 데이터를 totalList에 추가
        totalList.extend(li_list)

        
nameList = [] #매장명을 저장할 변수
arrList = [] #매장의 주소를 저장할 변수
tellList = [] #매장의 연락처를 저장할 변수        

#totalList의 개수만큼 반복을 시작한다.
#ltotalList의 데이터를 하나씩 빼서 infor라는 이름을 부여한다.
for infor in totalList:
    #매장 이름을 추출
    name = infor.find('strong').get_text()
    
    #추출한 매장 이름을 nameList에 추가
    nameList.append(name)
    
    #매장 주소와 연락처 정보를 추출
    arrAndTell = infor.find('p').get_text()
   
    #주소 정보만 추출
    arr = arrAndTell[:-9]
    
    #추출한 주소 정보를 arrList에 추가
    arrList.append(arr)
    
    #연락처 정보만 추출
    tell = arrAndTell[-9:]
    
    #추출한 연락처 정보를 tellList 추가
    tellList.append(tell)
    # -- for문 종료!
    
#스크래핑한 모든 정보를 하나의데이터의 저장
data = {
    '상호명':nameList
    , '주소':arrList
    , '연락처':tellList
}   

df = pd.DataFrame(data)

#엑셀 파일로 저장
df.to_excel('전국_스타벅스_매장정보.xlsx')

#완성된 출력
df    

C:\Users\admin\AppData\Local\Temp\ipykernel_6068\1364657220.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:/Users/admin/chromedriver_win32/chromedriver.exe")


,상호명,주소,연락처
0,역삼아레나빌딩,서울특별시 강남구 언주로 425 (역삼동),1522-3232
1,논현역사거리,서울특별시 강남구 강남대로 538 (논현동),1522-3232
2,신사역성일빌딩,서울특별시 강남구 강남대로 584 (논현동),1522-3232
3,국기원사거리,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232
4,대치재경빌딩R,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232
...,...,...,...
1788,세종청사,세종특별자치시 도움8로 81 (어진동),1522-3232
1789,세종아름,"세종특별자치시 아름서1길 13-9 (아름동) 103,104,105,202,203호",1522-3232
1790,세종한솔,세종특별자치시 노을3로 101 (한솔동),1522-3232
1791,세종소담,"세종특별자치시 소담3로 8 (소담동) 1동 12호,13호,14호,15호,15a호,16호",1522-3232
